## Import libraries and files

In [24]:
# Import the usual libraries

import pandas as pd
import math
import numpy as np
import seaborn as sns
import os
pd.set_option('display.max_columns',None)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

pd.set_option('mode.chained_assignment',None)

In [2]:
# Read our csv files

path = "D:\\My Documents\\Projects\\Automation ppt Caleb\\KPI original excel files\\"

dic = {}
list_col = []
for file in os.listdir(path):
    dic[file] = pd.read_csv(path+file)
    print(f"{file}: DONE   shape:{dic[file].shape}")

L'OREAL TR_KPI Dashboard December 2019 Final VF.csv: DONE   shape:(6817, 20)
L'OREAL TR_KPI Dashboard November 2019 Final VF.csv: DONE   shape:(5973, 20)
LOREAL LUXE_KPI Dashboard April 2019 final.csv: DONE   shape:(7759, 22)
LOREAL LUXE_KPI Dashboard August 2019 final.csv: DONE   shape:(7203, 20)
LOREAL LUXE_KPI Dashboard February 2019 final.csv: DONE   shape:(7168, 18)
LOREAL LUXE_KPI Dashboard January 2019 final.csv: DONE   shape:(4750, 17)
LOREAL LUXE_KPI Dashboard July 2019.csv: DONE   shape:(2677, 20)
LOREAL LUXE_KPI Dashboard Jun 2019.csv: DONE   shape:(6903, 20)
LOREAL LUXE_KPI Dashboard March 2019  final.csv: DONE   shape:(6542, 21)
LOREAL LUXE_KPI Dashboard May 2019 final.csv: DONE   shape:(5853, 20)
LOREAL LUXE_KPI Dashboard October 2019 Final.csv: DONE   shape:(5611, 20)
LOREAL LUXE_KPI Dashboard September 2019 Final.csv: DONE   shape:(5283, 20)


In [3]:
# Concat all the csv files into 1 dataframe

df_KPI = pd.concat(dic.values())

In [4]:
print(f"Our df_KPI dataframe has the following shape: {df_KPI.shape}")

Our df_KPI dataframe has the following shape: (72539, 23)


In [19]:
# A glance at the NaN values

df_isna = df_KPI.isna().sum().sort_values(ascending=False).reset_index()
df_isna.columns = ['column', 'count_nan']
df_isna['%'] = df_isna['count_nan']*100/df_KPI.shape[0]
px.bar(df_isna,
       x='column',
       y='%', 
       labels={'%':'Percentage of NaN value', 'column':''},
       title='Percentage of NaN value for each column', 
       height=900)

In [6]:
# Define df_KPI_rem as the dataframe containing the remarks/reasons of the delays

df_KPI_rem = df_KPI.loc[(df_KPI['Final Difference'].notnull()) 
                        & (df_KPI['Final Difference'] != 0) 
                        & (df_KPI['REMARKS'].notnull())]

In [7]:
# Harmonize the different remarks into clear clusters
# Any reason containing the string 'capacity' is categorized as 'capacity'
# Any reason containing the string 'documentation' is categorized as 'late documentation'
# Any reason containing the string 'cargo' is categorized as 'early cargo'
# Any reason containing the string 'vessel' is categorized as 'vessel'
# Any other reason is categorized as 'other'


df_KPI_rem.loc[df_KPI_rem['REMARKS'].str.lower().str.contains('capacity'),'remarks_cluster'] = 'capacity'

df_KPI_rem.loc[(df_KPI_rem['REMARKS'].str.lower().str.contains('documentation|purf')) 
               & ~(df_KPI_rem['REMARKS'].str.lower().str.contains('capacity')),'remarks_cluster'] = 'late documentation'

df_KPI_rem.loc[(df_KPI_rem['REMARKS'].str.lower().str.contains('cargo')) 
               & ~(df_KPI_rem['REMARKS'].str.lower().str.contains('capacity')),'remarks_cluster'] = 'early cargo'

df_KPI_rem.loc[(df_KPI_rem['REMARKS'].str.lower().str.contains('vessel')) 
               & ~(df_KPI_rem['REMARKS'].str.lower().str.contains('capacity')),'remarks_cluster'] = 'vessel'

df_KPI_rem.loc[~df_KPI_rem['REMARKS'].str.lower().str.\
               contains('capacity|documentation|cargo|vessel'),'remarks_cluster'] = 'other'

C:\Users\btg168\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\btg168\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
df_plot = (round(df_KPI_rem['remarks_cluster'].value_counts(normalize=True, dropna=False)*100,1)).reset_index()
df_plot.columns = ['Reason', 'Percentage']

In [9]:
px.bar(df_plot, 
       x='Reason', 
       y='Percentage',
       color='Reason', title='Percentage of delay reasons', text='Percentage')

In [25]:
df_plot_no_cargo = df_plot.loc[df_plot['Reason'] != 'early cargo']
df_plot_no_cargo['Percentage'] = round(df_plot_no_cargo['Percentage']*100/df_plot_no_cargo['Percentage'].sum(),1)
px.bar(df_plot_no_cargo, 
       x='Reason', 
       y='Percentage',
       color='Reason', 
       title='Percentage of real delay reasons', 
       text='Percentage')

### Breakdown of each remark cluster (Percentage of appearance)

In [11]:
# Breakdown for early cargo
df_KPI_rem.loc[df_KPI_rem['remarks_cluster']=='early cargo','REMARKS'].value_counts(normalize=True, dropna=False)

CARGO READY BEFORE COLLECTION DATE THEREFORE GI EARLIER                           0.657718
CARGO READY BEFORE COLLECTION DATE THEREFORE SHIP OUT EARLIER                     0.197507
TRANSPORTER NOT ABLE TO FULFILL DELIVERY.  CARGOES WILL BE DELIVERED ON MONDAY    0.072867
DOCUMENTATION SENT ON 21ST AUG, CARGO COLLECT ON 26TH AUG                         0.050815
CARGO READY BEFORE COLLECTION DATE THEREFORE GI EARLIER.                          0.011505
CARGO COMPLETED ONLY ON 04/02                                                     0.004794
CARGO COMPLETED ONLY ON 12/02                                                     0.004794
Name: REMARKS, dtype: float64

In [12]:
# Breakdown for capacity
df_KPI_rem.loc[df_KPI_rem['remarks_cluster']=='capacity','REMARKS'].value_counts(normalize=True, dropna=False)

DUE TO HUGE VOLUME AND OVERCAPACITY FOR THE WEEK, SHIPMENT HAS BEEN DELAYED                                   0.342752
Re-scheduling of cargo collection on 9th Sep due to warehouse capacity                                        0.119165
SHIPMENT DELAYED DELIVERY DUE TO WAREHOUSE CAPACITY ISSUE                                                     0.081081
WAREHOUSE CAPACITY                                                                                            0.061425
Delivery arranged on 30th Aug due to truck capacity constraint on 29th Aug                                    0.060197
DELIVERY TO FORWARDER ON 26TH JUN DUE TO CAPACITY ISSUE                                                       0.051597
WAREHOUSE CAPACITY ISSUE - DELIVERY  SCHEDULED ON 22ND AUG                                                    0.046683
FLIGHT SCHEDULED ON 27TH AUG DUE TO WAREHOUSE CAPACITY , CARGO TO BE COLLECTED ON 26TH AUG                    0.045455
WAREHOUSE CAPACITY ISSUE - DELIVERY TENTATIVELY 

In [13]:
# Breakdown for late documentation
df_KPI_rem.loc[df_KPI_rem['remarks_cluster']=='late documentation','REMARKS'].value_counts(normalize=True, dropna=False)

DOCUMENTATIONS WAS SUBMITTED LATE - PENDING FOR TRUCK CONFIRMATION    0.567823
DOCUMENTATIONS WAS SUBMITTED LATE - COLLECTION ON 16TH AUG            0.233438
Late Documentations To Forwarder By Bollore                           0.091483
Late Documentation from Bollore                                       0.059937
DOCUMENTATION SENT ON 21ST AUG, DELIVERY ON 22ND AUG                  0.047319
Name: REMARKS, dtype: float64

In [14]:
# Breakdown for vessel
df_KPI_rem.loc[df_KPI_rem['remarks_cluster']=='vessel','REMARKS'].value_counts(normalize=True, dropna=False)

VESSEL ETA SIN ON 18/01    0.513699
VESSEL ETA SIN ON 15/01    0.181507
VESSEL ETA SIN ON 19/04    0.119863
VESSEL ETA SIN ON 04/01    0.092466
VESSEL ETA SIN ON 06/03    0.034247
VESSEL ETA SIN ON 30/04    0.020548
VESSEL ETA SIN ON 03/05    0.010274
VESSEL ETA SIN ON 09/03    0.010274
VESSEL ETA SIN ON 16/01    0.006849
VESSEL ETA SIN ON 17/05    0.006849
VESSEL ETA SIN ON 07/05    0.003425
Name: REMARKS, dtype: float64

In [18]:
# Breakdown for other
df_KPI_rem.loc[df_KPI_rem['remarks_cluster']=='other','REMARKS'].value_counts(normalize=True, dropna=False)

PURF Submission delayed /  Collection on 27th Mar                                                                        0.732673
MIS-COMMUNICATION RESULTING IN NOT ABLE TO GET TRUCK FOR DELIVERY (REFER TO SHIPMENT DELAY COMPILATION FOR MORE INFO)    0.178218
UNABLE TO FULFILL ORDERS THEREFORE POSTPONE TO 10/05                                                                     0.049505
Under-estimation of shipment - unable to stuff in container, to arrange alternative flight                               0.019802
Delivery on 22nd Mar due to pending flight schedule                                                                      0.009901
AS PER YVONNE AND MANDY ADVISE TO POSTPONE SHIPMENT WITHOUT CHANGING RSD DATE                                            0.009901
Name: REMARKS, dtype: float64